El TARGET es el karma y los votos

- Predicción de los sentimientos en los comentarios publicados de Meneame , teniendo en cuenta el karma de la publicación y los votos recividos a la misma.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import nltk

In [2]:
df_completo = pd.read_csv("Data/100178_Comentarios.csv", sep=";")
df_completo.shape

(100178, 10)

In [5]:
df_completo.head()

,URL,fecha_envio,fecha_publicacion,pag_comentario,num_comentario,usuario,fecha,texto,votos,karma
0,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,1,ingenierodepalillos,1715371791,"O que les den de hostias hasta hartarse, son t...",8.0,77.0
1,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,2,Top_Banana,1715372035,"#1 Sea cual sea la opción, seguro que preferir...",3.0,35.0
2,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,3,Top_Banana,1715372235,"Un saludo Mysto, ¡hermoso!",4.0,51.0
3,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,4,CharlesBrowson,1715372556,Es tal es desproposito y la barbarie que no ca...,6.0,52.0
4,https://old.meneame.net/story/arresten-lideres...,1715371648,1715379303,1,5,BigMom,1715372955,"#1 Es el problema, uno no quiere rebajarse. Pe...",3.0,33.0


In [7]:
df_completo.shape

(100178, 10)

In [8]:

df_completo.isna().sum()

URL                    0
fecha_envio            0
fecha_publicacion      0
pag_comentario         0
num_comentario         0
usuario                2
fecha                  0
texto                 33
votos                785
karma                785
dtype: int64